# MSV / SS 2023 - Übung 7

## 7.1 Agreement

In [44]:
import pandas as pd
import numpy as np

#### Notation
- Menge von __Instanzen__ $\{ i | i \in I \}$ der Kardinalität $\mathbf{i}$
- Menge von __Kategorien__ $\{ k | k \in K \}$ der Kardinalität $\mathbf{k}$
- Menge von __Annotatoren__ (Codierern) $\{ c | c \in C \}$ der Kardinalität $\mathbf{c}$

- $\mathbf{n}_i^k$  ist die Anzahl der Annotatoren, die die Instanz $i$ der Kategorie $k$ zugeordnet haben
- $\mathbf{n}_c^k$ ist die Anzahl der Instanzen, die vom Annotatoren $c$ der Kategorie $k$ zugeordnet wurden 
- $\mathbf{n}^k$  ist die gesamte Anzahl der Instanzen, die der Kategorie $k$ zugeordnet wurden

#### Beispiel: Sentiment-Klassifikation (2 Mögliche Klassen: Positiv oder Negativ)

In [32]:
tuples = [[25, 15, 40],
         [5, 55, 60],
         [30, 70, 100]]
header = pd.MultiIndex.from_product([['Annotator A'], ['Pos', 'Neg', 'Total']])
arrays = [np.array(['Annotator B', 'Annotator B', 'Annotator B']), np.array(['Pos', 'Neg', 'Total'])]
sentiment = pd.DataFrame(tuples, index=arrays, columns=header,)

#### Konfusionsmatrix

In [23]:
sentiment

Annotator A          
                          Pos Neg Total
Annotator B Pos            25  15    40
            Neg             5  55    60
            Total          30  70   100

### Option 1: Percentage Agreement (Prozentuale Übereinstimmung)

Wieviel % der Labels wurden von beiden Annotator/innen vergeben?

$$ {{\mathbf{n}_{\textrm{agr}}} \over {\mathbf{n}}} $$

__Beobachtetes Übereinstimmung (observed agreement)__

arithmetisches Mittel aller $i$

$$ A_o = {1 \over \mathbf{i} } \sum_{i \in I} \textrm{agr}_i $$

- $\textrm{agr}_i$ ist
    - $0$ wenn die beiden Codierer Instanz $i$ verschiedenen Kategorie zugeordnet haben
    - $1$ wenn die beiden Codierer Instanz $i$ der gleichen Kategorie zugeordnet haben
    

$$ A_o = {1 \over 100 } (25 + 55) = 0.8 $$ 

__Nachteile:__
- $A_o$ berücksichtigt nicht den Anteil an Übereinstimmung, der durch __Zufall__ erreicht wurde (chance agreement)
- Chance agreement häng  sehr stark von der jeweiligen Annotationsaufgabe ab (Anzahl an Kategorien & höhere Übereinstimmung für die dominante Klasse) 

__Die Beobachtete Übereinstimmung muss mithilfe der zufällige Übereinstimmung korrigiert werden__

### Option 2: zufalls-korrigierte Koeffizient  $\kappa$ (Cohen 1960): 
    
(1) Welche Übereinstimmung erwarten wir?

$A_e$

(2) Welche Übereinstimmung über Zufall hinaus
wurde tatsächlich gefunden?

$A_o - A_e$

(3) Welche Übereinstimmung über Zufall hinaus war überhaupt möglich? 

$1 - A_e$

(4) Welche Übereinstimmung wurde erreicht, basierend auf (1) und (2)?

$${A_o - A_e \over 1 - A_e}$$

- Wenn die Übereinstimmung $= 0$: zufällige Übereinstimmung (keine Übereinstimmung über $A_e$ hinaus)
- If agreement $= 1$: bestmögliche Übereinstimmung (höheste Übereinstimmung über $A_e$ hinaus)
- If agreement $< 0$: schlechter als der Zufall

(5) Berechnung der $A_e$ mittels $P(k|c_1)$ - die Wahrscheinlichkeit, dass Annotator $c$ eine beliebige Instanz als Kategorie $k$ annotiert (Unabhängigkeitsannahme)

$$A^\kappa_e = \sum_{k \in K} P(k|c_1) \cdot P(k|c_2)$$

- Individuelle Verteilung für jede Annotatorin: zufällige Zuweisung von Kategorien zu Instanzen wird von A-priori-Wahrscheinlichkeit gesteuert, die für jede Annotatorin bestimmt wird 

$$\hat{P}(k|c_i) = { \mathbf{n}_{c_ik} \over \mathbf{i} }$$

$$A^\kappa_e = \sum_{k \in K} \hat{P}(k|c_1) \cdot \hat{P}(k|c_2) = \sum_{k \in K} { \mathbf{n}_{c_1k} \over \mathbf{i} } \cdot { \mathbf{n}_{c_2k} \over \mathbf{i} } $$

In [24]:
sentiment

Annotator A          
                          Pos Neg Total
Annotator B Pos            25  15    40
            Neg             5  55    60
            Total          30  70   100

$$\hat{P}(\textrm{Pos}|\textrm{AnnotatorA}) = 30 / 100 = 0.3$$
$$\hat{P}(\textrm{Pos}|\textrm{AnnotatorB}) = 40 / 100 = 0.4$$

$$\hat{P}(\textrm{Neg}|\textrm{AnnotatorA}) = 70 / 100 = 0.7$$
$$\hat{P}(\textrm{Neg}|\textrm{AnnotatorB}) = 60 / 100 = 0.6$$

$$A^\kappa_e = \hat{P}(\textrm{Pos}|\textrm{AnnotatorA}) \cdot \hat{P}(\textrm{Pos}|\textrm{AnnotatorB}) + 
\hat{P}(\textrm{Neg}|\textrm{AnnotatorA}) \cdot \hat{P}(\textrm{Neg}|\textrm{AnnotatorB}) = 0.54$$

$$\kappa = {0.8-0.54 \over 1 - 0.54} \approx 0.56$$

### Mehr als 2 Annotatoren?

In [30]:
tuples = [["2", "1"],
         ["0", "3"],
         [" ", " "],
         ["1", "2"],
         ["90 (0.3)", "210 (0.7)"]]
header = ['Pos', 'Neg']
arrays = ['Rew1', 'Rew2', '...', 'Rew100', 'Total']
sentiment2 = pd.DataFrame(tuples, index=arrays, columns=header,)

#### Übereinstimmungstabelle

In [31]:
sentiment2

,Pos,Neg
Rew1,2,1
Rew2,0,3
...,,
Rew100,1,2
Total,90 (0.3),210 (0.7)


$\kappa$ kann generalisiert werden

#### Observed Agreement

$A_o$: Anteil an übereinstimmenden Beurteilungspaaren ${\mathbf{n}_{ik} \choose 2}$ aus der Gesamtanzahl an Beurteilungspaaren für diese Instanz ${\mathbf{c} \choose 2}$

$$\textrm{agr}_1 = {1 \over {\mathbf{c} \choose 2}} \sum_{k \in K}{\mathbf{n}_{ik} \choose 2} = {1 \over \mathbf{c}(\mathbf{c} - 1)}\sum_{k \in K}\mathbf{n}_{ik}(\mathbf{n}_{ik} - 1)$$

$$A_o = {1 \over \mathbf{i}} \sum_{i \in I} \textrm{agr}_i = {1 \over \mathbf{ic}(\mathbf{c} -1)}\sum_{i \in I}\sum_{k \in K}\mathbf{n}_{ik}(\mathbf{n}_{ik} - 1)$$

#### für Rew_1:

$$\mathbf{n}_{\textrm{Utt}_{1}\textrm{Pos}} = 2$$
$$\mathbf{n}_{\textrm{Utt}_{1}\textrm{Neg}} = 1$$
    
$$\textrm{agr}_i = {1 \over {3 \choose 2}} \bigg({\mathbf{n}_{\textrm{Rew}_{1}\textrm{Pos}} \choose 2} + {\mathbf{n}_{\textrm{Rew}_{1}\textrm{Neg}} \choose 2} \bigg) = {1 \over 3} (1 + 0) \approx 0.33$$
    


#### Expected Agreement

$A_e$: Wahrscheinlichkeit, dass 2 zufällig ausgewählte
Annotatoren einer Instanz zufällig die gleiche Kategorie zuweisen)

- Multi$-\kappa$: 
    - __Separate Wahrscheinlichkeitsverteilung für Annotatoren und Kategorien___
    
    $$\hat{P}(k) = { 1 \over \mathbf{ic} } \mathbf{n}_ck$$

    (joint probability of each coder making this assignment independently)

    $$A^\kappa_e = \sum_{k \in K} {1 \over {\mathbf{c} \choose 2}} \sum_{m = 1}^{\mathbf{c}-1} \sum_{n = m + 1}^{\mathbf{c}} \hat{P}(k|c_m) \cdot \hat{P}(k|c_n)$$

#### Krippendorff's  $\alpha$

- kann auf mehr als 2 Annotatoren angewendet werden
- misst die fehlende Übereinstimmung, oder Disagreement (nicht Agreement)
 $$\alpha = 1 - {D_o \over D_e}$$
- bezieht mit ein, wie schwerwiegend die fehlende Übereinstimmung ist
- kann mit fehlenden Werten umgehen


- https://repository.upenn.edu/cgi/viewcontent.cgi?article=1043&context=asc_papers 
- https://www.nltk.org/_modules/nltk/metrics/agreement.html

In [33]:
from nltk.metrics.agreement import AnnotationTask

In [60]:
matrix = [[0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
         [1, 1, 1, 0, 0, 1, 0, 0, 0, 0]]
header = pd.Series(range(1,11))
arrays = ['meg', 'owen']
caseA = pd.DataFrame(matrix, index=arrays, columns=header,)
caseA

,1,2,3,4,5,6,7,8,9,10
meg,0,1,0,0,0,0,0,0,1,0
owen,1,1,1,0,0,1,0,0,0,0


In [61]:
caseA_tuples = caseA.reset_index().melt(id_vars=['index'])
caseA_tuples.columns = ["coder", "item", "code"]
caseA_tuples = list(zip(caseA_tuples.coder, caseA_tuples.item, caseA_tuples.code))
caseA_tuples

[('meg', 1, 0),
 ('owen', 1, 1),
 ('meg', 2, 1),
 ('owen', 2, 1),
 ('meg', 3, 0),
 ('owen', 3, 1),
 ('meg', 4, 0),
 ('owen', 4, 0),
 ('meg', 5, 0),
 ('owen', 5, 0),
 ('meg', 6, 0),
 ('owen', 6, 1),
 ('meg', 7, 0),
 ('owen', 7, 0),
 ('meg', 8, 0),
 ('owen', 8, 0),
 ('meg', 9, 1),
 ('owen', 9, 0),
 ('meg', 10, 0),
 ('owen', 10, 0)]

In [62]:
task = AnnotationTask(data=caseA_tuples)

In [63]:
task.avg_Ao()

0.6

In [64]:
task.kappa()

0.09090909090909075

In [65]:
task.alpha()

0.09523809523809512

## 7.2 Named Entity Recognition mit SpaCy (Chap 3)

basiert auf: 
- Duygu Altinok, Mastering Spacy, Chapt 3 - Code und Daten: https://github.com/PacktPublishing/Mastering-spaCy/tree/main/Chapter03 & 
- Duygu Altinok, Mastering Spacy, Chapt 4 - Code und Daten: https://github.com/PacktPublishing/Mastering-spaCy/tree/main/Chapter04

In [66]:
import re
import spacy
nlp_en = spacy.load("en_core_web_md") 

Jede Instanz der Klasse ``Language`` enthält eine sprachspezifische Verarbeitungspipeline

<img src="https://spacy.io/images/pipeline.svg" alt="SpaCy architecture" width="500" />

### Entitäts vorhersagen (NER)

In [68]:
doc = nlp_en("Citing high fuel prices, United Airlines said Friday it has increased fares by $6 per round trip on flights to some cities also served by lower-cost carriers. American Airlines, a unit of AMR Corp., immediately matched the move, spokesman Tim Wagner said. United, a unit of UAL Corp., said the increase took effect Thursday and applies to most routes where it competes against discount carriers, such as Chicago to Dallas and Denver to San Francisco.")

doc.ents

(United Airlines,
 Friday,
 6,
 American Airlines,
 AMR Corp.,
 Tim Wagner,
 United,
 UAL Corp.,
 Thursday,
 Chicago,
 Dallas,
 Denver,
 San Francisco)

In [52]:
for token in doc:
    print(token.text, token.ent_type_)

Citing 
high 
fuel 
prices 
, 
United ORG
Airlines ORG
said 
Friday DATE
it 
has 
increased 
fares 
by 
$ 
6 MONEY
per 
round 
trip 
on 
flights 
to 
some 
cities 
also 
served 
by 
lower 
- 
cost 
carriers 
. 
American ORG
Airlines ORG
, 
a 
unit 
of 
AMR ORG
Corp. ORG
, 
immediately 
matched 
the 
move 
, 
spokesman 
Tim PERSON
Wagner PERSON
said 
. 
United ORG
, 
a 
unit 
of 
UAL ORG
Corp. ORG
, 
said 
the 
increase 
took 
effect 
Thursday DATE
and 
applies 
to 
most 
routes 
where 
it 
competes 
against 
discount 
carriers 
, 
such 
as 
Chicago GPE
to 
Dallas GPE
and 
Denver GPE
to 
San GPE
Francisco GPE
. 


In [53]:
from spacy import displacy
displacy.render(doc, style="ent")

In [69]:
spacy.explain("GPE")

'Countries, cities, states'

#### NER auf Deutsch

In [71]:
nlp_de = spacy.load("de_core_news_md")

In [72]:
doc = nlp_de('Das von Einheimischen und Urlaubern befürchtete große Chaos \
            auf Sylt blieb am langen Pfingstwochenende aus. Die zuständige Polizeidirektion \
            in Flensburg hat Bilanz gezogen: "Für uns war es ein ruhiges Pfingstwochenende, \
            vergleichbar mit solchen von Vor-Corona-Zeiten", sagt Sprecherin Sandra Otte NDR \
            Schleswig-Holstein. Laut Bilanz der Polizei kamen viele partyfreudige Menschen nach Sylt, \
            darunter auch etwa 150 Personen, die der Punker-Szene angehören. Insgesamt gab es laut \
            Otte zwar viele Einsätze, aber das sei normal, wenn viel gefeiert werde.')

from spacy import displacy
displacy.render(doc, style="ent")

#### Warum ist die Erkennung von Entitäten auf Deutsch anspruchsvoller?

### Ein Beispiel mit Web-Scraping

Also bitte vorher New ‣ Terminal auswahlen und 
```
pip install beautifulsoup4
pip install html5lib
```

In [73]:
from bs4 import BeautifulSoup
import requests

In [74]:
def url_text(url_string):

    res = requests.get(url_string)

    html = res.text

    soup = BeautifulSoup(html, 'html')

    for script in soup(["script", "style", 'aside']):

        script.extract()

    text = soup.get_text()

    return " ".join(text.split())

dw_art = url_text("https://www.dw.com/en/nato-chief-talks-sweden-finland-membership-with-erdogan/a-62034347")


In [75]:
dw_art

'NATO chief talks Sweden, Finland membership with Erdogan – DW – 06/04/2022 You need to enable JavaScript to run this app. Skip to contentSkip to main menuSkip to more DW sitesLatest videosLatest audioRegionsAfricaAsiaEuropeLatin AmericaMiddle EastNorth AmericaGermanyTopicsClimateEqualityHealthHuman RightsMigrationTechnologyCategoriesBusinessScienceEnvironmentCultureSportsLive TVLatest audioLatest videosIn focusRussia\'s war in UkraineDiversitySweden and Finland applied to join the Western defence alliance last month in response to Russia\'s invasion of UkraineImage: Sascha Steinach/IMAGOPoliticsTurkeyNATO chief talks Sweden, Finland membership with Erdogan06/04/2022June 4, 2022The talks represent the latest attempt to work past the roadblock Ankara has put up over Sweden and Finland joining the military alliance.https://p.dw.com/p/4CHxzAdvertisementNATO Secretary-General Jens Stoltenberg said Saturday he had held a "constructive phone call" with Turkish President Recep Tayyip Erdogan 

In [76]:
doc = nlp_en(dw_art)
len(doc.ents)

151

In [77]:
from collections import Counter

labels = [ent.label_ for ent in doc.ents]

Counter(labels)

Counter({'ORG': 35,
         'GPE': 59,
         'PERSON': 16,
         'DATE': 13,
         'NORP': 10,
         'CARDINAL': 9,
         'PRODUCT': 2,
         'LOC': 2,
         'TIME': 4,
         'LAW': 1})

#### Was sagen Entitäten über den Text aus?

In [78]:
items = [ent.text for ent in doc.ents]

Counter(items).most_common(10)

[('Finland', 13),
 ('NATO', 11),
 ('Sweden', 11),
 ('Turkey', 8),
 ('Ankara', 5),
 ('US', 4),
 ('DW', 3),
 ('Russia', 3),
 ('Erdogan', 2),
 ('last month', 2)]

In [79]:
from spacy import displacy
displacy.render(doc, style="ent")

### Regeln-basierte Suche nach Entitäten

Wir inisialisieren ein Matcher-Objekt mit den Wortschatz des Sprachmodells

In [81]:
from spacy.matcher import Matcher

doc = nlp_en("Good morning, I want to reserve a ticket. I will then say good evening!")

matcher = Matcher(nlp_en.vocab)

Wir definieren zwei Pattern (1 List-Item = 1 Token)

In [82]:
pattern1 = [{"LOWER": "good"}, {"LOWER": "morning"}, {"IS_PUNCT": True}]

matcher.add("morningGreeting", [pattern1])

pattern2 = [{"LOWER": "good"}, {"LOWER": "evening"}, {"IS_PUNCT": True}]

matcher.add("eveningGreeting",  [pattern2])

Wir suchen nach der Pattern

In [83]:
matches = matcher(doc)

for match_id, start, end in matches:

     m_span = doc[start:end]  

     print(start, end, m_span.text)

0 3 Good morning,
14 17 good evening!


### Lexika-basierte Suche nach Entitäten

Wir inisialisieren ein PhraseMatcher-Objekt mit den Wortschatz des Sprachmodells

In [38]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp_en.vocab)

Wir definieren ein Lexikon

In [39]:
terms = ["Angela Merkel", "Donald Trump", "Alexis Tsipras"]

Wir suchen nach der Entitäten in einem Text

In [40]:
patterns = [nlp_en.make_doc(term) for term in terms]

matcher.add("politiciansList", patterns)

doc = nlp_en("3 EU leaders met in Berlin. German chancellor Angela Merkel first welcomed \
             the US president Donald Trump. The following day Alexis Tsipras joined \
             them in Brandenburg.")

matches = matcher(doc)

for mid, start, end in matches:

    print(start, end, doc[start:end])

9 11 Angela Merkel
17 19 Donald Trump
23 25 Alexis Tsipras


### Regelbasierte Suche nach IPs

Hier möchten wir IP-Nummern als Entitäten extrahieren. Mit "Shape" brauchen wir nur ein paar Beispiele bieten. 

In [41]:
matcher = PhraseMatcher(nlp_en.vocab, attr="SHAPE") # "Shape" erlaubt uns, die 'Form' des Musters zuverwenden (einfacher als RegEx)

ip_nums = ["127.0.0.0", "127.256.0.0"]

patterns = [nlp_en.make_doc(ip) for ip in ip_nums]

matcher.add("IPNums", patterns)

In [42]:
doc = nlp_en("This log contains the following IP addresses: 192.1.1.1 and 192.12.1.1 and 192.160.1.1 .")

for mid, start, end in matcher(doc):

    print(start, end, doc[start:end])

8 9 192.1.1.1
12 13 192.160.1.1


### EntityRuler

Wir können wir regel-basierte Entitätserkennung mit maschinellen Lernen kombinieren?
Entity Ruler kann als neue Komponente der Pipeline hinzugefügt werden

In [46]:
doc = nlp_en("I have an acccount with chime since 2017")
print(doc.ents)

(2017,)


In [47]:
patterns = [{"label": "ORG", "pattern": [{"LOWER": "chime"}]}]
ruler = nlp_en.add_pipe("entity_ruler")
ruler.add_patterns(patterns)
doc2 = nlp_en("I have an acccount with chime since 2017")

print(doc2.ents)
print(doc2[5].ent_type_)

(chime, 2017)
ORG


## Hausaufgaben

### Übung 7.1

Annotieren Sie die Daten im 'reviews_toannotate.csv' (mit "pos" oder "neg" in der in der letzten Spalte (csv mit ";" als delimiter) und und berechnen Sie die Agreement (kappa und alpha) mit der bereits kommentierten Datei 'reviews_gold.csv'. 

In [113]:
import pandas as pd
from nltk.metrics.agreement import AnnotationTask

In [117]:
annotator1_data = pd.read_csv('reviews_gold.csv',delimiter=';', header=None)
print(annotator1_data)

                                                    0    1
0   an entertaining 2 hours awaits the audience in...  pos
1   suddenly warren beatty doesn't look so absurd ) .  neg
2     by the time they show you , it doesn't matter .  neg
3   deniro is also affable , tough on the outside ...  pos
4   its 100-minute running time feels like 100 yea...  neg
5   this film would definitely not have been as go...  pos
6   because of freeman's powerful presence , he ca...  pos
7   every shot is choreographed as a portrait -- a...  pos
8   in short , rudolph has created a world that it...  pos
9   kick boxing deaths and conflicted nuns are not...  neg
10  the costumes are remarkable and have jean-paul...  pos
11  i was surprised how expressive an actor he pro...  pos
12  and if i spoiled it for you , good , i saved y...  neg
13  i like his quick-edit style , because it's abr...  pos
14  the problems in logic are flaws , but don't ru...  pos
15  connery plays his character with the exact amo...  p

In [123]:
matrix = [annotator1_data[1],
         annotator1_data[1]] # replace with your data
header = pd.Series(range(1,40))
arrays = ['gold', 'you']
annotation = pd.DataFrame(matrix, index=arrays, columns=header,)
annotation

,1,2,3,4,5,6,7,8,9,10,...,30,31,32,33,34,35,36,37,38,39
gold,neg,neg,pos,neg,pos,pos,pos,pos,neg,pos,...,neg,pos,neg,pos,neg,neg,neg,neg,pos,neg
you,neg,neg,pos,neg,pos,pos,pos,pos,neg,pos,...,neg,pos,neg,pos,neg,neg,neg,neg,pos,neg


In [130]:
annotation_tuples = annotation.reset_index().melt(id_vars=['index'])
annotation_tuples.columns = ["coder", "item", "code"]
annotation_tuples = list(zip(annotation_tuples.coder, annotation_tuples.item, annotation_tuples.code))
annotation_tuplestask = AnnotationTask(data=annotation_tuples)

### Übung 7.2

Finden Sie die Entitäten in diesem Text mit Hilfe des Spacy-NER-Modus. Suchen Sie nach möglichen Fehlern und überlegen Sie, wie Sie diese beheben können.

In [90]:
harrypotter_file = 'harrypotter.txt'
harrypotter_txt = open(harrypotter_file).read()
# replace \n with whitespace
harrypotter_txt = harrypotter_txt.replace('\n',' ')
harrypotter_txt

'"Nobody was very attentive in lessons, being much more interested in the arrival that evening of the people from Beauxbatons and Durmstrang; even Potions was more bearable than usual, as it was half an hour shorter.  When the bell rang early, Harry, Ron, and Hermione hurried up to Gryffindor Tower, deposited their bags and books as they had been instructed, pulled on their cloaks, and rushed back downstairs into the entrance hall. The Heads of Houses were ordering their students into lines.  "Weasley, straighten your hat," Professor McGonagall snapped at Ron.  "Miss Patil, take that ridiculous thing out of your hair."  Parvati scowled and removed a large ornamental butterfly from the end of her plait.  "Follow me, please," said Professor McGonagall.  "First years in front. . . no pushing.. ."  They filed down the steps and lined up in front of the castle. It was a cold, clear evening; dusk was falling and a pale, transparent-looking moon was already shining over the Forbidden Forest. 